# IOHprofiler Hands-on Session - GECCO '20

[Hao Wang](https://www.lip6.fr/actualite/personnes-fiche.php?ident=D2381), [Diederick Vermetten](https://www.universiteitleiden.nl/en/staffmembers/diederick-vermetten), [Carola Doerr](https://webia.lip6.fr/~doerr/), [Ofer M. Shir](https://ofersh.github.io/telhai/), and [Thomas Bäck](https://www.universiteitleiden.nl/en/staffmembers/thomas-back)

<br>


The plan is to guide you through the basica usage of **IOHexperimenter** and **IOHanalyzer**.

## General Information

* __Wiki page__: [https://iohprofiler.github.io](https://iohprofiler.github.io)
* __General Documentation__: [https://arxiv.org/abs/1810.05281](https://arxiv.org/abs/1810.05281)
* __IOHanalyzer Documentation__: [https://arxiv.org/abs/2007.03953](https://arxiv.org/abs/2007.03953)
* __IOHanalyzer Online Service__: [http://iohprofiler.liacs.nl](http://iohprofiler.liacs.nl)

## Part-I: **IOHexperimenter** (Python Interface)

To get started, we need to install the `IOHexperimenter` package from `pip` (you only need to run the cell below only once): 

In [ ]:
!pip install IOHexperimenter
!rm -rf data/

     |████████████████████████████████| 1.0MB 4.8MB/s 


Importing the package:


In [ ]:
# Import the IOH_function class
from IOHexperimenter import IOH_function

Checking the `docstring` of `IOH_function`:

In [ ]:
# View docstring of IOH_function
?IOH_function

Yes, you could directly use the well-known [BBOB problem set](https://github.com/numbbo/coco):

In [ ]:
# Create a function object, either by giving the function id from within the suite
f = IOH_function(fid = 7, dim = 5, iid = 1, suite = 'BBOB')

In [ ]:
# Or by giving the function name
f2 = IOH_function(fid = "Sphere", dim = 5, iid = 1)

In [ ]:
# Acces properties of the function
f.number_of_variables

5

The search domain is set to $[-5, 5]^d$ by default, as suggested in BBOB.

In [ ]:
f.lowerbound, f.upperbound

(array([-5., -5., -5., -5., -5.]), array([5., 5., 5., 5., 5.]))

In [ ]:
f.final_target_hit

False

In [ ]:
# Evaluate the origin...
f([0,0,0,0,0])

106.05513987471636

### Running an algorithm

A simple random-search example wich accepts an argument of class IOH_function.

In [ ]:
import numpy as np

In [ ]:
def random_search(func, param_1 = None, budget = None):
    if budget is None:
        budget = int(func.number_of_variables * 1e4)

    f_opt = np.Inf
    x_opt = None

    for i in range(budget):
        # sampling a new point u.a.r. in the search domain
        x = np.random.uniform(func.lowerbound, func.upperbound)
        f = func(x)
        if f < f_opt:
            f_opt = f
            x_opt = x
    return f_opt, x_opt

To record data, we need to add a logger to the problem

In [ ]:
from IOHexperimenter import IOH_logger

In [ ]:
?IOH_logger

In [ ]:
logger = IOH_logger(location = "data", foldername = "run", 
                    name = "random_search", 
                    info = "test of IOHexperimenter in python")

This can then be attached to the problem

In [ ]:
f.add_logger(logger)

Now, we can run the algorithm and store data:

In [ ]:
random_search(f)

(94.15646842857485,
 array([-0.26558154,  0.74860565,  0.35503907,  4.24473928,  3.21339203]))

To ensure all data is written, we should clear the logger after running our experiments:

In [ ]:
f.clear_logger()

And our benchmark data is already in place:

In [ ]:
!ls -lha data/
!echo
!cat data/run/IOHprofiler_f7_Step_Ellipsoid.info
!echo
!echo
!head -n 10 data/run/data_f7_Step_Ellipsoid/IOHprofiler_f7_DIM5.dat

total 20K
drwxr-xr-x 1 root root 4.0K Jul  9 11:05 .
drwxr-xr-x 1 root root 4.0K Jul  9 09:53 ..
drwxr-xr-x 1 root root 4.0K Jun 26 16:26 .config
drwxr-xr-x 3 root root 4.0K Jul  9 11:05 data
drwxr-xr-x 1 root root 4.0K Jun 26 16:26 sample_data

suite = "No suite", funcId = 7, funcName = "Step_Ellipsoid", DIM = 5, maximization = "F", algId = "random_search", algInfo = "test of IOHexperimenter in python"
%
data_f7_Step_Ellipsoid/IOHprofiler_f7_DIM5.dat, 1:35673|1.21647
"function evaluation" "current f(x)" "best-so-far f(x)" "current af(x)+b"  "best af(x)+b"
1 766.466 766.466 859.406 859.406
2 239.636 239.636 332.576 332.576
10 204.033 204.033 296.973 296.973
13 191.236 191.236 284.176 284.176
19 57.1758 57.1758 150.116 150.116
37 22.4291 22.4291 115.369 115.369
513 13.9905 13.9905 106.931 106.931
876 10.713 10.713 103.653 103.653
1161 9.56859 9.56859 102.509 102.509


### Tracking Dynamic Parameters

If we want to track parameters of the algorithm, we need to slightly restructure it by turning it into a class, where the variables we want totrack are properties thereof:

In [ ]:
class random_search_class(object):
    def __init__(self, budget):
        self.budget = int(budget)
        self.f_opt = np.Inf
        self.x_opt = None

    def __call__(self, func):
        for i in range(self.budget):
            x = np.random.uniform(func.lowerbound, func.upperbound)
            f = func(x)
            if f < self.f_opt:
                self.f_opt = f
                self.x_opt = x
        return self.f_opt, self.x_opt
    
    # A mockup internal parameter to track
    @property
    def param_rate(self):
        return np.random.randint(100)


# create the optimizer object
opt = random_search_class(1e4)

We could then use the function `logger.track_parameters` of the logger to register the parameter to be tracked

In [ ]:
logger = IOH_logger(location = "data", foldername = "parameter-tracking", 
                    name = "random_search", 
                    info = "test of IOHexperimenter in python")

logger.track_parameters(algorithm = opt, parameters = 'param_rate')
f.add_logger(logger) # IMPORTANT! don't forget to attach the logger to the test function again..

In [ ]:
opt(f)
f.clear_logger()  # Remember to flush the data to files..

You could see now there is an extra column "`param_rate`" in the raw data file.

In [ ]:
!ls -lha data/
!echo
!cat data/parameter-tracking/IOHprofiler_f7_Step_Ellipsoid.info
!echo
!echo
!head -n 10 data/parameter-tracking/data_f7_Step_Ellipsoid/IOHprofiler_f7_DIM5.dat

total 16K
drwxr-xr-x 4 root root 4.0K Jul  9 11:09 .
drwxr-xr-x 1 root root 4.0K Jul  9 11:05 ..
drwx------ 3 root root 4.0K Jul  9 11:09 parameter-tracking
drwx------ 3 root root 4.0K Jul  9 11:05 run

suite = "No suite", funcId = 7, funcName = "Step_Ellipsoid", DIM = 5, maximization = "F", algId = "random_search", algInfo = "test of IOHexperimenter in python"
%
data_f7_Step_Ellipsoid/IOHprofiler_f7_DIM5.dat, 1:58121|2.5681

"function evaluation" "current f(x)" "best-so-far f(x)" "current af(x)+b"  "best af(x)+b" "param_rate"
50001 461.376 1.21647 554.316 94.1565 64
50002 314.185 1.21647 407.125 94.1565 11
50006 78.779 1.21647 171.719 94.1565 60
50015 62.6312 1.21647 155.571 94.1565 78
50030 46.4346 1.21647 139.375 94.1565 7
50031 42.1045 1.21647 135.044 94.1565 85
50087 19.5027 1.21647 112.443 94.1565 88
50307 15.152 1.21647 108.092 94.1565 73
51030 9.98858 1.21647 102.929 94.1565 48


In [ ]:
# it is always good to check the docstring...
?logger.track_parameters

### Using the `IOHexperimenter` object

Alternatively, we can use the `IOHexperimenter` class to easily run the benchmarking in multiple functions, and to specify which parameters to track.

In [ ]:
from IOHexperimenter import IOHexperimenter

In [ ]:
?IOHexperimenter

This can be initialized to a suite (`PBO` or `BBOB` are available) by providing lists of function ids (or names), dimensions, instance ids, and a number of independent repetitions as follows:

In [ ]:
exp = IOHexperimenter()
exp.initialize_BBOB(fids = [1,2], dims = [5], iids = [1,2], repetitions = 2)

There is also the option to provide a set of standard `python` functions to the experimenter object:

In [ ]:
?exp.initialize_custom

For now, we stick with the BBOB suite. We can set up the logging as follows:

In [ ]:
exp.set_logger_location(location = "data_test", foldername = "run")

The algorithm we want to run has property `param_rate`, so we can enable the tracking using the function below:

In [ ]:
exp.set_parameter_tracking("param_rate")

In [ ]:
# it is always good to check the docstring...
?exp.set_parameter_tracking

We can parallel the experimentation if the sequential execution is taking way too long..

In [ ]:
?exp.set_parallel

The data storage settings are also customizable:

In [ ]:
?exp.set_logger_options

Finally, we can run the experiment by simply providing the experimenter object with a list of algorithms to run

In [ ]:
# just a mockup code. In reality, of course you want to add multiple different algorithms..
exp([opt(1000)] * 2)

Running single-threaded
1 1000 0.00015138508252931606
1 1000 1.9075929859439873e-06
1 1000 0.0406816466179649
1 1000 0.038245274507711306
2 1000 6.346088751509518e-07
2 1000 6.32270121217581e-06
2 1000 6.800983388507359
2 1000 4.8374851495284235


[None, None, None, None]

## Part-II: **IOHanalyzer** Hands-on

We have to options to use the analyzer: 

* to use the online serive directly: [http://35.241.182.66](http://35.241.182.66) (the data sets are going to use is already uploaded here), which is hosted on Google Cloud exclusively for this session (you do not recommend to visit [http://iohprofiler.liacs.nl/](http://iohprofiler.liacs.nl/) since this is hosted mererly on a single-CPU machine..)
* to install it locally on your machine:
  1. You need to install `R` first. Please download the correct installation file accorinding to your operating system (Windows, MacOS, Linux..):   [https://cran.r-project.org/](https://cran.r-project.org/)
  2. After installation, please start a `R` console and install the IOHanalzyer package by: 
  ```{r, eval = FALSE}
  R> install.packages('IOHanalzyer')
  ```
  3.Then you could start the GUI (grahical user interface) using the following command (a new tab should be open in your browser automatiocally):
  ```{r, eval = FALSE}
  R> runServer()
  ```
  4. For the hands-on session, some example data sets can be downloaded from here: [https://github.com/IOHprofiler/IOHdata](https://github.com/IOHprofiler/IOHdata). Please create the folder called `repository` (`~/repository` on Linux and MacOS and `C:\Users\username\repository`) in your home directory and unbzip the data set into it (IOHanalyzer will scan this folder later).
  
### Exemplary Usage

Throughout this exercise, a data set called `2019gecco-ins1-11run` will be used and is already hosted for the online version. For the local usage, it can be downloaded from [(https://github.com/IOHprofiler/IOHdata](https://github.com/IOHprofiler/IOHdata):

1. Please navigate to the **Data Upload** section and select the required data set in **Load Data from Repository** box. 

2. Now please navigate thought **Fixed-Target Results** $\rightarrow$ **Single Function** $\rightarrow$ **Data Summary**. Please try to figure out the following question using the table provided in this subsection:

    * What is the largest observed budget value across all algorithms in $625$D on function $f1$?
    * For $625$D and function $f19$, what is the number of runs that algorithm $(1+1) \text{ EA}_{>0}$ hits its best function value?
    * For $64$D and function $f21$, what is the _worst recorded function value_ of algorithm gHC (greedy Hill-Climber)? What is the _worst reached function value_ then? and what is the difference between those two values?
    * For $100$D and function $f23$, what are the mean, median, standard deviation and _expected running time_ (ERT) of algorithm $(1+1)$ fGA (fast GA) at target function value $-1$? What is the ERT and success rate of this algorithm at its best achieved function value?
    * For $625$D and function $f4$, what is the running time sample of algorithm RLS (Randomized Local Search) at its best achieved function value?
  
3. Next, please select the **Probability Mass Function** subsection, and use it to answer the following questions:

    * For function $f17$ in $625D$, please create a figure showing histograms of the number of function evaluations needed to reach a target value of $400$ for all variants of the $(1+10)$ EA-variants. Please ensure that the bin sizes allow for an easy visual comparison among the different subplots. Please download the resulting figure in svg-format.
    * Using the same function and algorithms, please create a plot showing the probability mass functions of the required number of function evaluations needed to reach a target of $450$. Please make sure that the individual runtime samples are shown clearly in the figure, scale the y-axis logarithmically and download the resulting plot as a pdf.
  
4. Please navigate thought **Fixed-Budget Results** $\rightarrow$ **Single Function** $\rightarrow$ **Data Summary**. Please try to solve the following questions using the functionality available in this tab:

    * What is the budget used for the $(30,30)$ vGA on function $f5$ in $100D$?
    * For function $f19$ on $100D$, what is the mean target hit for the RLS algorithm after $100$ function evaluations? How does this compare to the other algorithms? Download a csv-table showing this comparison.
    * For function $f2$ on $16D$, how many algorithms have hit a mean target value of $16$ after $1000$ function evaluations? Try to download a table showing only the data for these algorithms in a tex-format.
    * For function $f17$ on $100D$, what are the individual target values hit after $1$, $15\,001$, $30\,001$, $45\,001$ and $60\,001$ function evaluations respectively. Please download a table showing this information.
